In [1]:
import AdventOfCode
import qualified Data.Vector as V

In [2]:
input <- dayString 7

instructions = read ("[" ++ input ++ "]") :: [Int]

In [3]:
data Opcode = Opcode
    { opcodeCode  :: Int
    , opcodeMode1 :: Bool
    , opcodeMode2 :: Bool
    , opcodeMode3 :: Bool
    } deriving (Eq, Show)

In [4]:
parseCode :: Int -> Opcode
parseCode i = let
    m3   = i `div` 10000
    m2   = (i - m3*10000) `div` 1000
    m1   = (i - m3*10000 - m2*1000) `div` 100
    code = i `rem` 100
    in Opcode code (m1==1) (m2==1) (m3==1)

In [5]:
step :: V.Vector Int -> [Int] -> [Int] -> Int -> (V.Vector Int, [Int], [Int], Int)
step program input output index = do
    let opcode = parseCode (program V.! index)
    case opcodeCode opcode of
        1 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            p = program V.// [(o,a+b)]
            in (p, input, output, index+4)
        2 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            p = program V.// [(o,a*b)]
            in (p, input, output, index+4)
        3 -> let
            v = head input
            input' = tail input
            o = program V.! (index+1)
            p = program V.// [(o, v)]
            in (p, input', output, index+2)
        4 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            output' = a:output
            in (program, input, output', index+2)
        5 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            in if a/=0 then (program, input, output, b) else (program, input, output, index+3)
        6 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            in if a==0 then (program, input, output, b) else (program, input, output, index+3)
        7 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            in if a < b then (program V.// [(o,1)], input, output, index+4) else (program V.// [(o, 0)], input, output, index+4)
        8 -> let
            a = if opcodeMode1 opcode then program V.! (index+1) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then program V.! (index+2) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            in if a == b then (program V.// [(o,1)], input, output, index+4) else (program V.// [(o, 0)], input, output, index+4)
        99 -> (program, input, output, index)

Line 6: Reduce duplication
Found:
a = if opcodeMode1 opcode then program V.! (index + 1) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then program V.! (index + 2) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:12:13Line 6: Reduce duplication
Found:
a = if opcodeMode1 opcode then program V.! (index + 1) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then program V.! (index + 2) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:36:13Line 6: Reduce duplication
Found:
a = if opcodeMode1 opcode then program V.! (index + 1) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then program V.! (index + 2) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:41:13

In [6]:
loop program input output index = let
    (program', input', output', index') = step program input output index
    in if index == index' then (program', output', index) else loop program' input' output' index'

In [7]:
run input = let (_, o, _) = loop (V.fromList instructions) input [] 0 in o

[28]

In [8]:
import Data.List
import Data.Function.Memoize
import Data.Function

In [9]:
run' = memoize run

In [10]:
chain = foldl' (\value n -> run' (n:value))

In [11]:
maximumBy (compare `on` fst) $ map (\p -> (head $ chain [0] p, p)) $ permutations [0..4]

(75228,[0,3,4,2,1])